In [1]:
import os
import gc
import random

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt

import bloscpack as bp

from tsfresh.feature_extraction import feature_calculators

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold
from sklearn.metrics import f1_score

from NNs import WaveTRSFM_Classifier

In [2]:
DEVICE = 'cuda:0'
EPOCHS = 96
BATCHSIZE = 32
SEED = 19550423
LR = 0.0005
SPLITS = 5

torch.manual_seed(SEED)

In [3]:
trn_fs = sorted([f for f in os.listdir('../input/feats_srs') if (('trn_srs_dat' in f) and ('s500' in f) and ('w500' in f) and ('basicwithnew2' in f))])
lbl_fs = sorted([f for f in os.listdir('../input/feats_srs') if ('trn_srs_lbl' in f) and ('s500' in f) and ('w500' in f) and ('basicwithnew2' in f)])

tst_fs = sorted([f for f in os.listdir('../input/feats_srs') if (('tst_srs_dat' in f) and ('s500' in f) and ('w500' in f) and ('basicwithnew2' in f))])
tst_fs = [tst_fs[i] for i in [0, 11, 12, 13, 14, 15, 16, 17, 18, 19]] + tst_fs[1:11]

In [4]:
series_trn = np.concatenate(
    [bp.unpack_ndarray_from_file(os.path.join('../input/feats_srs', f)) for f in trn_fs],
    axis=0
)

series_lbl = [bp.unpack_ndarray_from_file(os.path.join('../input/feats_srs', f)) for f in lbl_fs]

series_grp = np.concatenate(
    [np.ones(shape=(arr.shape[0],)) * i for i, arr in zip([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], series_lbl)],
    axis=0
).astype(int)

series_lbl = np.concatenate(
    series_lbl,
    axis=0
)[:, :, None]

series_tst = np.concatenate(
    [bp.unpack_ndarray_from_file(os.path.join('../input/feats_srs', f)) for f in tst_fs],
    axis=0
)


In [5]:
lbl_skewness = [feature_calculators.binned_entropy(lst, max_bins=20) for lst in series_trn[:, :, 0].tolist()]
lbl_skewness = pd.qcut(pd.Series(lbl_skewness), q=10, duplicates='drop')

skf_grp = [str(a) + '_' + str(b) for a, b in zip(series_grp, lbl_skewness)]
us = np.unique(skf_grp)
umap = {u: i for u, i in zip(us, range(len(us)))}
skf_grp = [umap[u] for u in skf_grp]

In [6]:
for i in range(series_trn.shape[-1]):

    avg = series_trn[:, :, i].mean()
    std = series_trn[:, :, i].std()
    series_trn[:, :, i] = (series_trn[:, :, i] - avg) / std
    series_tst[:, :, i] = (series_tst[:, :, i] - avg) / std
    
    #print('---------')
    #print('{:d} - max {:.3f}; min {:.3f};'.format(i, series_trn[:, :, i].max(), series_trn[:, :, i].min()))
    #print('{:d} - max {:.3f}; min {:.3f};'.format(i, series_tst[:, :, i].max(), series_tst[:, :, i].min()))
    print('progress: {:02d} / {:02d}; '.format(i+1, series_trn.shape[-1]), end='\r')

In [7]:
class Waveset(Dataset):
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        
        if self.labels is None:
            return data.astype(np.float32)
        else:
            labels = self.labels[idx]
            return (data.astype(np.float32), labels.astype(int))

In [8]:
def fold_train_validate(
    model, optimizer, criterion, scheduler,
    training_loader, validation_loader, fold_number,
    save_path='./saved_models/wavenet_model_fold{:03d}_checkpoint.pth',
    early_stopping=15,
):

    trn_losses = [np.nan]
    vld_losses = [np.nan]
    vld_f1s = [np.nan]
    
    last_best = 0

    for epc in range(EPOCHS):
        print('===========================================================')

        epoch_trn_losses = []
        epoch_trn_lbls = []
        epoch_trn_prds = []
        epoch_vld_losses = []
        epoch_vld_lbls = []
        epoch_vld_prds = []

        # ------ training ------
        model.train()
        for i, (trn_batch_dat, trn_batch_lbl) in enumerate(training_loader):
            trn_batch_dat, trn_batch_lbl = trn_batch_dat.to(DEVICE), trn_batch_lbl.to(DEVICE)

            optimizer.zero_grad()
            trn_batch_prd = model(trn_batch_dat)
            trn_batch_prd = trn_batch_prd.view(-1, trn_batch_prd.size(-1))
            trn_batch_lbl = trn_batch_lbl.view(-1)
            loss = criterion(trn_batch_prd, trn_batch_lbl)
            loss.backward()
            optimizer.step()

            epoch_trn_losses.append(loss.item())
            epoch_trn_lbls.append(trn_batch_lbl.detach().cpu().numpy())
            epoch_trn_prds.append(trn_batch_prd.detach().cpu().numpy())

            print(
                'Epoch {:03d}/{:03d} - Training batch {:04d}/{:04d}: Training loss {:.6f};'.format(
                    epc + 1, EPOCHS, i + 1, len(training_loader), epoch_trn_losses[-1],
                ), 
                end='\r'
            )

        # ------ validation ------
        model.eval()
        with torch.no_grad():
            for i, (vld_batch_dat, vld_batch_lbl) in enumerate(validation_loader):
                vld_batch_dat, vld_batch_lbl = vld_batch_dat.to(DEVICE), vld_batch_lbl.to(DEVICE)

                vld_batch_prd = model(vld_batch_dat)
                vld_batch_prd = vld_batch_prd.view(-1, vld_batch_prd.size(-1))
                vld_batch_lbl = vld_batch_lbl.view(-1)
                loss = criterion(trn_batch_prd, trn_batch_lbl)

                epoch_vld_losses.append(loss.item())
                epoch_vld_lbls.append(vld_batch_lbl.detach().cpu().numpy())
                epoch_vld_prds.append(vld_batch_prd.detach().cpu().numpy())

                print(
                    'Epoch {:03d}/{:03d} - Validation batch {:04d}/{:04d}: Validation loss {:.6f};'.format(
                        epc + 1, EPOCHS, i + 1, len(validation_loader), epoch_vld_losses[-1],
                    ), 
                    end='\r'
                )

        # ------ epoch end ------
        f1_trn = f1_score(
            np.concatenate(epoch_trn_lbls, axis=0), 
            np.concatenate(epoch_trn_prds, axis=0).argmax(1),
            labels=list(range(11)), 
            average='macro'
        )
        f1_vld = f1_score(
            np.concatenate(epoch_vld_lbls, axis=0), 
            np.concatenate(epoch_vld_prds, axis=0).argmax(1),
            labels=list(range(11)), 
            average='macro'
        )


        print(
            'Epoch {:03d}/{:03d} - Mean training loss {:.6f}; Mean training F1 {:.6f}; Mean validation loss {:.6f}; Mean validation F1 {:.6f}; Learning rate {:.6f};'.format(
                epc + 1, EPOCHS, np.mean(epoch_trn_losses), f1_trn, np.mean(epoch_vld_losses), f1_vld, scheduler.get_lr()[0],
            )
        )

        if f1_vld > np.nanmax(vld_f1s):
            torch.save(
                {
                    'epoch': epc + 1,
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'f1': f1_vld,
                    'loss': np.mean(epoch_vld_losses),
                }, 
                save_path.format(fold_number)
            )
            
            last_best = epc
            
        if epc - last_best > early_stopping:
            break

        vld_f1s.append(f1_vld)

        scheduler.step()

In [9]:
N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, random_state=SEED)

In [10]:
for fld, (ndcs_trn, ndcs_vld) in enumerate(skf.split(series_trn, skf_grp)):
    print('################################################################')
    print('Training/validation for fold {:d}/{:d};'.format(fld+1, N_FOLDS))
    
    # setup fold data
    dat_trn, lbl_trn = series_trn[ndcs_trn], series_lbl[ndcs_trn]
    dat_vld, lbl_vld = series_trn[ndcs_vld], series_lbl[ndcs_vld]
    
    waveset_trn = Waveset(dat_trn, lbl_trn)
    waveset_vld = Waveset(dat_vld, lbl_vld)

    loader_trn = DataLoader(waveset_trn, BATCHSIZE, shuffle=True, num_workers=2, pin_memory=True)
    loader_vld = DataLoader(waveset_vld, BATCHSIZE, shuffle=False, num_workers=2, pin_memory=True)
    
    # setup fold model
    mdl = WaveTRSFM_Classifier(series_trn.shape[-1]).to(DEVICE)
    critrn = nn.CrossEntropyLoss()
    optimzr = torch.optim.AdamW(mdl.parameters(), lr=LR)
    schdlr = torch.optim.lr_scheduler.CosineAnnealingLR(optimzr, T_max=EPOCHS, eta_min=LR/100)
    
    # run
    fold_train_validate(
        model=mdl, optimizer=optimzr, criterion=critrn,
        scheduler=schdlr, training_loader=loader_trn, validation_loader=loader_vld,
        fold_number=fld,
        save_path='./saved_models/waveTRSFM_model_basicwithnew2_fold{:03d}_checkpoint.pth',
        early_stopping=25
    )
    #break

################################################################
Training/validation for fold 1/5;
Epoch 001/096 - Mean training loss 0.427053; Mean training F1 0.719997; Mean validation loss 0.157788; Mean validation F1 0.904941; Learning rate 0.000500;
Epoch 002/096 - Mean training loss 0.159833; Mean training F1 0.882014; Mean validation loss 0.146290; Mean validation F1 0.914156; Learning rate 0.000500;
Epoch 003/096 - Mean training loss 0.133551; Mean training F1 0.897157; Mean validation loss 0.134820; Mean validation F1 0.924612; Learning rate 0.000499;
Epoch 004/096 - Mean training loss 0.178729; Mean training F1 0.875723; Mean validation loss 0.163564; Mean validation F1 0.927975; Learning rate 0.000498;
Epoch 005/096 - Mean training loss 0.113420; Mean training F1 0.916307; Mean validation loss 0.087535; Mean validation F1 0.933290; Learning rate 0.000497;
Epoch 006/096 - Mean training loss 0.103833; Mean training F1 0.923584; Mean validation loss 0.115553; Mean validation F1

In [15]:
submission = pd.read_csv('../input/sample_submission.csv', dtype={'time': str, 'open_channels': 'Int64'})

In [17]:
submission_pred = np.zeros(shape=(submission.shape[0], 11))

waveset_tst = Waveset(series_tst)
loader_tst = DataLoader(waveset_tst, BATCHSIZE, shuffle=False, num_workers=2, pin_memory=True)

for fld in range(5):
    print('-------- fold {:d} --------'.format(fld))
    fld_weight = torch.load('./saved_models/waveTRSFM_model_basicwithnew2_fold{:03d}_checkpoint.pth'.format(fld))
    print('model validation loss: {:.3f}; validation f1: {:.3f};'.format(fld_weight['loss'], fld_weight['f1']))
    mdl = WaveTRSFM_Classifier(series_trn.shape[-1]).to(DEVICE)
    mdl.load_state_dict(fld_weight['model'])
    mdl.eval()
    with torch.no_grad():
        tst_fold_prd = []
        for tst_batch_dat in loader_tst:
            tst_batch_prd = mdl(tst_batch_dat.to(DEVICE))
            tst_batch_prd = tst_batch_prd.view(-1, tst_batch_prd.size(-1)).detach().cpu().numpy()
            tst_fold_prd.append(tst_batch_prd)
            
        submission_pred += np.concatenate(tst_fold_prd, 0)

-------- fold 0 --------
model validation loss: 0.095; validation f1: 0.939;
-------- fold 1 --------
model validation loss: 0.062; validation f1: 0.937;
-------- fold 2 --------
model validation loss: 0.095; validation f1: 0.937;
-------- fold 3 --------
model validation loss: 0.061; validation f1: 0.939;
-------- fold 4 --------
model validation loss: 0.061; validation f1: 0.939;


In [18]:
submission['open_channels'] = submission_pred.argmax(1)
submission.to_csv("../submissions/sub0_waveTRSFM_basicwithnew2_cvbyentropy_meanstdnorm.csv", index=False)